# Failure Pareto for Test Steps
This notebook creates a failure pareto report for test steps. It ties into the **Test Monitor Service** for retrieving filtered test results and steps, the **Notebook Execution Service** for running outside of Jupyterhub, and the **Test Monitor Reports page** at #testmonitor/reports for displaying results.

The parameters and output use a schema recognized by the Test Monitor Reports page, which can be implemented by various report types. The Failure Pareto notebook produces data that is best shown in a pareto chart.

### Imports
Import Python modules for executing the notebook. Pandas is used for building and handling dataframes. Scrapbook is used for recording data for the Notebook Execution Service. The SystemLink Test Monitor Client provides access to test result data for processing.

In [1]:
import pandas as pd
import scrapbook as sb

import systemlink.clients.nitestmonitor as testmon

### Parameters
- `results_filter`: Dynamic Linq query filter for test results from the Test Monitor Service; for the Test Steps Failure Pareto, it is recommended to provide program names 
  Options: Any valid Test Monitor Results Dynamic Linq filter
  Default: `'startedWithin <= "30.0:0:0"'`
- `group_by`: The dimension along which to reduce; what each bar in the output graph represents  
  Options: Path  
  Default: Path

Parameters are also listed in the metadata for the parameters cell, along with their default values. The Notebook Execution services uses that metadata to pass parameters from the Test Monitor Reports page to this notebook. Available `group_by` options are listed in the metadata as well; the Test Monitor Reports page uses these to validate inputs sent to the notebook.

To see the metadata, select the code cell and click the wrench icon in the far left panel.

In [2]:
results_filter = 'workspaceName = "EPL Manufacturing" && startedWithin <= "4"'
products_filter = ''
group_by = 'Path'

### Mapping from grouping options to Test Monitor terminology
Translate the grouping options shown in the Test Monitor Reports page to keywords recognized by the Test Monitor API.

In [3]:
groups_map = {
    'Path': 'path'
}
grouping = groups_map[group_by]

### Create Test Monitor client
Establish a connection to SystemLink over HTTP.

In [4]:
api_client = testmon.ApiClient()
results_api = testmon.ResultsApi(api_client)
steps_api = testmon.StepsApi(api_client)

### Query for results
Query the Test Monitor Service for results matching the `results_filter` parameter. Then query for all test steps in those results.

In [5]:
async def perform_batched_query(query_function, query, response_field):
    results = []

    response = await query_function(post_body=query)
    while response.continuation_token:
        results = results + getattr(response, response_field)
        query.continuation_token = response.continuation_token
        response = await query_function(post_body=query)
    return results

results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter, order_by=testmon.ResultField.STARTED_AT, take=1000
)

results = await perform_batched_query(results_api.query_results_v2, results_query, 'results')
results_list = [result.to_dict() for result in results]
#display(results_list)

#Get the unique (latest) SN and id that fail
formatted_results = {
    'id': [result['id'] for result in results_list],
    'serial_number': [result['serial_number'] for result in results_list],
    'status': [result['status']['status_type'] for result in results_list],
    'started_at': [result['started_at'] for result in results_list]
}
df_results = pd.DataFrame.from_dict(formatted_results)
df_results_unique = df_results.groupby('serial_number').last().reset_index()
df_results_unique = df_results_unique.loc[df_results_unique['status'] == "FAILED"].reset_index(drop=True)


result_ids = []
steps = []
if results:
    steps_filter = ''
    for index,result in enumerate (df_results_unique['id']):
        if steps_filter != '':
            steps_filter += ' || '
        steps_filter += 'resultId == "' + df_results_unique['id'][index] + '"' 

    if steps_filter:
        steps_filter = f'status.statusType == "FAILED" && ({steps_filter})'
    else:
        steps_filter = 'status.statusType == "FAILED"'

    steps_query = testmon.StepsAdvancedQuery(
        steps_filter, order_by=testmon.StepField.STARTED_AT, take=1000
    )
    steps = await perform_batched_query(steps_api.query_steps_v2, steps_query, 'steps')

steps_list = [step.to_dict() for step in steps]

#display(steps_list)

### Get group names
Collect the group name for each result based on the `group_by` parameter.

In [6]:
group_names = []
for step in steps_list:
    if grouping in step:
        group_names.append(step[grouping])

### Create pandas dataframe
Put the data into a dataframe whose columns are test result id, status, and group name.

In [14]:
formatted_results = {
    'id': [step['step_id'] for step in steps_list],
    'parent_id': [step['parent_id'] for step in steps_list],
    'status': [step['status']['status_type'] if step['status'] else None for step in steps_list],
    grouping: group_names
}

df_results = pd.DataFrame.from_dict(formatted_results)
df_parent_ids = df_results[['parent_id']].copy().drop_duplicates()
df_results = df_results[~df_results['id'].isin(df_parent_ids['parent_id'])]
if df_results['path'].any():
    df_results['path'] = df_results['path'].str.replace(r'^root\.MainSequence Callback\.|^root\.', '')
    df_results['path'] = df_results['path'].str.partition('\n')[2]
    
display(df_results)

,id,parent_id,status,path
0,60d15548ca744a3c64c03f22,60d15548ca744a3c64c03f14,FAILED,Delta\nDelta.S5 (T1-T2)
1,60d15549ca744a3c64c03f2e,60d15545ca744a3c64c03ed5,FAILED,Algorithm Result
2,60d15d55ca744a3c64c050ed,60d15d54ca744a3c64c050df,FAILED,Delta\nDelta.S5 (T1-T2)
3,60d15d55ca744a3c64c050f9,60d15d50ca744a3c64c050a0,FAILED,Algorithm Result
6,206071,190561,FAILED,Call Product TSM\n3.9 - TD Frequency Test
...,...,...,...,...
328,60d28841ca744a3c64c12589,60d2883cca744a3c64c12530,FAILED,Algorithm Result
329,60d2884dca744a3c64c12655,60d2884cca744a3c64c12647,FAILED,Delta\nDelta.S5 (T1-T2)
330,60d2884dca744a3c64c12661,60d28849ca744a3c64c12608,FAILED,Algorithm Result
331,60d28858ca744a3c64c1272d,60d28857ca744a3c64c1271f,FAILED,Delta\nDelta.S5 (T1-T2)


### Aggregate results into groups
Aggregate the data for each unique group and status.

*See documentation for [size](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.size.html) and [unstack](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) here.*

In [8]:
df_results.fillna(value='', inplace=True)
df_grouped = df_results.groupby([grouping, 'status']).size().unstack(fill_value=0)
if 'PASSED' not in df_grouped:
    df_grouped['PASSED'] = 0
if 'FAILED' not in df_grouped:
    df_grouped['FAILED'] = 0
if 'ERRORED' not in df_grouped:
    df_grouped['ERRORED'] = 0

### Failure Pareto calculation
Count the number of test step failures and calculate cumulative values for the pareto.

In [9]:
df_fail_count = pd.DataFrame(df_grouped['FAILED'] + df_grouped['ERRORED'])
if grouping != 'started_at':
    df_fail_count.sort_values(by=[0], ascending=False, inplace=True)
total = df_fail_count[0].sum()
pareto_values = []
cumulative = 0
for data_member in df_fail_count[0]:
    cumulative += data_member
    pareto_values.append(100 * (cumulative / total))
df_fail_count = df_fail_count[df_fail_count[0] != 0]

df_pareto = df_fail_count.reset_index().set_axis([grouping, 'fail_count'], axis=1)
df_pareto['cumulative'] = pareto_values

### Convert the dataframe to the SystemLink reports output format
The result format for a SystemLink report consists of a list of output objects as defined below:
- `type`: The type of the output. Accepted values are 'data_frame' and 'scalar'.
- `id`: Corresponds to the id specified in the 'output' metadata. Used for returning multiple outputs with the 'V2' report format.
- `data`: A dict representing the 'data_frame' type output data.
    - `columns`: A list of dicts containing the names and data type for each column in the dataframe.
    - `values`: A list of lists containing the dataframe values. The sublists are ordered according to the 'columns' configuration.
- `value`: The value returned for the 'scalar' output type.
- `config`: The configurations for the given output.
    - `title`: The output title.
    - `graph`: The graph configurations.
        - `axis_labels`: The x-axis label and y-axis label.
        - `plots`: A list of plots to display mapped from the dataframe's columns, along with configuration options.
            - `x`: The dataframe column corresponding to the x-axis values.
            - `y`: The dataframe column corresponding to the y-axis values.
            - `style`: The plot's style. Accepted values are ['LINE', 'BAR', 'SCATTER'].
            - `color`: The plot's color. Accepted formats are ['blue', '#0000ff', 'rbg(0,0,255)'].
            - `label`: The plot's name, to be shown in a plot legend. 
            - `secondary_y`: Whether or not to display this plot on a second y-axis.
            - `group_by`: A list of columns in the dataframe on which to group data, e.g. to color individual points.
        - `orientation`: 'HORIZONTAL' or 'VERTICAL'.
        - `stacked`: Whether or not to display the plots stacked on top of each other.

Here is an example of a notebook result with two outputs, one of which is a dataframe with two columns, and the other is a scalar value:
```
[{
    'type': 'data_frame',
    'id': 'output_id_1',
    'data': {
        'columns': [
            {'name': 'time', 'type': 'datetime'},
            {'name': 'value', 'type': 'number'}
         ],
        'values': [
            ['2020-09-29T00:00:00.000Z', 46.1538461538],
            ['2020-09-30T00:00:00.000Z', 63.1578947368],
            ...
         ]
    },
    'config': {
        'title': 'My Title',
        'graph': {
            'axis_labels': ['X Axis', 'Y Axis'],
            'orientation': 'VERTICAL',
            'plots': [
                {'x': 'time', 'y': 'value', 'style': 'BAR', 'color': '#0000ff', 'label': 'Plot 1'}
            ]
        }
    }
}, {
    'type': 'scalar',
    'id': 'output_id_2',
    'config': {
        'title': 'My Title'
    },
    'value': 5
}]
```

For this report, there is one output, which is a dataframe with three columns. The first column contains the group categories, which are the step paths by default. The second column contains failure counts, and the third column contains the cumulative totals as percents.

| part_number | fail_count | cumulative |
|-------------|------------|------------|
| 151837H     | 102        | 34.459459  |
| 154261F     | 98         | 67.567568  |
| 193343E     | 96         | 100        |

The graph configuration specifies two plots: A bar chart for the failure counts and a line chart for the cumulative percentage. We use Pandas to convert the dataframe built in the previous cells into a tabular format and then return that with the result object.

In [10]:
df_pareto[grouping].replace(r'^$', 'No ' + group_by, regex=True, inplace=True)

df_dict = {
    'columns': pd.io.json.build_table_schema(df_pareto, index=False)['fields'],
    'values': df_pareto.values.tolist(),
}

pareto_graph = {
    'type': 'data_frame',
    'id': 'failure_pareto_steps_graph',
    'data': df_dict,
    'config': {
        'title': 'Failure Pareto - Steps by {}'.format(group_by),
        'graph': {
            'axis_labels': [group_by, 'Failure Count', 'Cumulative %'],
            'plots': [
                {'x': grouping, 'y': 'fail_count', 'style': 'BAR', 'group_by': [grouping]},
                {'x': grouping, 'y': 'cumulative', 'secondary_y': True, 'style': 'LINE'}
            ],
            'orientation': 'VERTICAL'
        }
    }
}

result = [pareto_graph]

### Record results with Scrapbook

In [11]:
sb.glue('result', result)